In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_keys

#Configure gmaps
gmaps.configure(api_key=g_keys)

In [2]:
vacation_df= pd.read_csv("weather_json.csv")
vacation_df.head